In [0]:
import numpy as np
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, LongType
import random

In [0]:
from pulp import PULP_CBC_CMD, HiGHS_CMD, LpVariable, LpBinary, LpInteger, LpProblem, LpMinimize, lpSum, value, HiGHS, LpStatus

In [0]:
import pyspark.sql.functions as f

In [0]:
values = [
    100, 95, 80, 90, 20, 15, 60, 55, 10, 50,
    33, 50, 61, 89, 81, 88, 100, 48, 25, 66,
    33, 42, 68, 85, 46, 21, 67, 86, 13, 58,
    34, 30, 42, 56, 31, 50, 99, 27, 57, 81
]

In [0]:
n = 40 #number of products
a = 50
b = 100
#stock = [np.random.randint(a,b) for p in range(0, n)] #how many units are available, ranging between 50 and 100
stock = values
prods = [f"p{i}" for i in range(0,n)] ##create n products
a = 0.2 
b = 20
weights = np.random.uniform(a, b, n)

In [0]:
data = [(p,g, float(round(w,2))) for p,g,w in zip(prods,stock, weights)]

In [0]:
schema = StructType([
    StructField("Product_id", StringType(), True),
    StructField("Stock", IntegerType(), True),
    StructField("Weights", FloatType(), True)
])

df_prods = spark.createDataFrame(data, schema=schema)

In [0]:
df_prods.show(5)

+----------+-----+-------+
|Product_id|Stock|Weights|
+----------+-----+-------+
|        p0|   55|   1.64|
|        p1|   95|  18.76|
|        p2|   60|  10.49|
|        p3|   67|   5.34|
|        p4|   64|   5.31|
+----------+-----+-------+
only showing top 5 rows


In [0]:
n = 81 ##number of slots
slots = list(np.arange(n)) 
corridor1 = [1 for c in range(27)]
corridor2 = [2 for c in range(27)]
corridor3 = [3 for c in range(27)]
#corridor4 = [4 for c in range(15)]
corridors = corridor1 + corridor2 + corridor3 #+ corridor4   
len(corridors)

81

In [0]:
capactity = [200 for p in range(n)]

In [0]:
nums = list(np.arange(1,32))
skip = [10, 11, 21, 22]
nums = [n for n in range(1, 32) if n not in skip]
distances = [n for n in nums for _ in range(3)]
len(distances)

81

In [0]:
nums1 = list(np.arange(4,7))
nums2 = list(np.arange(3,6))
nums3 = list(np.arange(1,4))
ergonomy = nums1*9 + nums2*9 + nums3*9

81

In [0]:
hei = list(np.arange(0,3))
height = hei*27
len(height)

81

In [0]:
data2 = [(s, cor, cap, dist, erg, hei) for s,cor,cap, dist, erg, hei in zip(slots,corridors,capactity,distances,ergonomy,height)]

In [0]:
schema2 = StructType([
    StructField("Slot", IntegerType(), True),
    StructField("Corridor", IntegerType(), True),
    StructField("Capacity", IntegerType(), True),
    StructField("Distance", IntegerType(), True),
    StructField("Ergonomy", IntegerType(), True),
    StructField("Height", IntegerType(), True)])



In [0]:
df_slots = spark.createDataFrame(data2, schema=schema2)
df_slots.show(5)

+----+--------+--------+--------+--------+------+
|Slot|Corridor|Capacity|Distance|Ergonomy|Height|
+----+--------+--------+--------+--------+------+
|   0|       1|     200|       1|       4|     0|
|   1|       1|     200|       1|       5|     1|
|   2|       1|     200|       1|       6|     2|
|   3|       1|     200|       2|       4|     0|
|   4|       1|     200|       2|       5|     1|
+----+--------+--------+--------+--------+------+
only showing top 5 rows


In [0]:
df_slots.groupBy('Corridor').count().show()

+--------+-----+
|Corridor|count|
+--------+-----+
|       1|   27|
|       2|   27|
|       3|   27|
+--------+-----+



In [0]:
combo1 = [random.choice(prods) for p in range(8)]
combo2 = [random.choice(prods) for p in range(8)]
cols = ['p1', 'p2']
data = [(c1,c2) for c1,c2 in zip(combo1, combo2)]
df_combos = spark.createDataFrame(data, cols)
df_combos.show(5)

+---+---+
| p1| p2|
+---+---+
|p33| p7|
|p11|p30|
|p25|p38|
|p17| p1|
| p9|p34|
+---+---+
only showing top 5 rows


In [0]:
#####SOLVING THE PROBLEM

In [0]:
df_slots.show(5)

+----+--------+--------+--------+--------+------+
|Slot|Corridor|Capacity|Distance|Ergonomy|Height|
+----+--------+--------+--------+--------+------+
|   0|       1|     200|       1|       4|     0|
|   1|       1|     200|       1|       5|     1|
|   2|       1|     200|       1|       6|     2|
|   3|       1|     200|       2|       4|     0|
|   4|       1|     200|       2|       5|     1|
+----+--------+--------+--------+--------+------+
only showing top 5 rows


In [0]:
products = [r["Product_id"] for r in df_prods.select("Product_id").collect()]
slots    = [r["Slot"] for r in df_slots.select("Slot").collect()]

In [0]:
df_slots.show(5)

+----+--------+--------+--------+--------+------+
|Slot|Corridor|Capacity|Distance|Ergonomy|Height|
+----+--------+--------+--------+--------+------+
|   0|       1|     200|       1|       4|     0|
|   1|       1|     200|       1|       5|     1|
|   2|       1|     200|       1|       6|     2|
|   3|       1|     200|       2|       4|     0|
|   4|       1|     200|       2|       5|     1|
+----+--------+--------+--------+--------+------+
only showing top 5 rows


In [0]:
units = {r['Product_id']: float(r['Stock']) for r in df_prods.select('Product_id', 'Stock').collect()}
weight = {r['Product_id']: float(r['Weights']) for r in df_prods.select('Product_id', 'Weights').collect()}
distance = {r['Slot']: float(r['Distance']) for r in df_slots.select('Slot', 'Distance').collect()}
ergonomy = {r["Slot"]: float(r["Ergonomy"]) for r in df_slots.select('Slot', 'Ergonomy').collect()}
height = {r["Slot"]: float(r["Height"]) for r in df_slots.select('Slot', 'Height').collect()}
corridor = {r["Slot"]: float(r["Corridor"]) for r in df_slots.select('Slot', 'Corridor').collect()}
capacity = {r['Slot']: float(r['Capacity']) for r in df_slots.select('Slot', 'Capacity').collect()}


In [0]:
df_slots.groupBy('Corridor').count().show()

+--------+-----+
|Corridor|count|
+--------+-----+
|       1|   27|
|       2|   27|
|       3|   27|
+--------+-----+



In [0]:
# Calculate neighbors
slots_by_corredor = (
    df_slots
    .groupBy("Corridor")
    .agg(f.collect_list("Slot").alias("slots"))
    .collect()
)

neighbors = {}
for row in slots_by_corredor:
    corredor_val = row["Corridor"]
    slots_corredor = sorted(row["slots"])
    for i in range(0, len(slots_corredor), 9): # each slot has 9 neighbors
        bloco = slots_corredor[i:i+9]
        for s in bloco:
            neighbors[s] = [v for v in bloco if v != s]

In [0]:
avaiable_corridors = sorted(set(corridor.values()))

In [0]:
# Model
model = LpProblem("Slotting", LpMinimize)

## variables
y = {(p, s): LpVariable(f"y_{p}_{s}", cat=LpBinary) for p in products for s in slots}
x = {(p, s): LpVariable(f"x_{p}_{s}", lowBound=0, cat=LpInteger) for p in products for s in slots}
z = {p: LpVariable(f"z_{p}", lowBound=0) for p in products}

In [0]:
## regularizing parameters for distance and ergonomy
theta = 10
mu = 10

model += (
    lpSum([y[p, s] * (distance[s] * units[p]) for p in products for s in slots]) +
    theta * lpSum([x[p, s] * weight[p] * ergonomy[s] for p in products for s in slots]) +
    mu * lpSum([x[p, s] * weight[p] * height[s] for p in products for s in slots])
)


In [0]:
print(f'We have {len(model.variables())} variables!') ## up to now, we have n_prods*n_slots*2 variables (the z variables is not considered yet)

We have 6480 variables!


In [0]:
# Constraints
for p in products:
    for s in slots:
        model += weight[p] * x[p, s] <= capacity[s] * y[p, s]
        model += y[p, s] <= x[p, s]

for p in products:
    model += lpSum([x[p, s] for s in slots]) == units[p]

for s in slots:
    model += lpSum([y[p, s] for p in products]) <= 1

for p in products:
    model += lpSum([y[p, s] for s in slots]) <= np.ceil(units[p] * weight[p] / capacity[slots[0]])

for p in products:
    for s1 in slots:
        for s2 in slots:
            if s1 >= s2:
                continue
            if s2 not in neighbors.get(s1, []):
                model += y[p, s1] + y[p, s2] <= 1


In [0]:
M = 3 #any big number
combo_produtos_df = df_combos.select("p1").union(
    df_combos.select("p2").withColumnRenamed("p2", "p1")
).distinct()

combo_produtos = [row["p1"] for row in combo_produtos_df.collect()]

for p in products:
    if p in combo_produtos:
        for s in slots:
            cs = corridor[s]
            model += z[p] - cs <= M * (1 - y[p, s])
            model += z[p] - cs >= -M * (1 - y[p, s])

for row in df_combos.select("p1", "p2").collect():
    p1 = row["p1"]
    p2 = row["p2"]
    model += z[p1] == z[p2]

In [0]:
##Now, the numbers of variables and constraints are
print("Number of variables:", len(model.variables()))
print("Number of constraints:", len(model.constraints))

Number of variables: 6494
Number of constraints: 125557


In [0]:
solver_list = pulp.listSolvers(onlyAvailable=True)
print(solver_list)

['PULP_CBC_CMD', 'HiGHS']


In [0]:
# Solve
model.solve(HiGHS(msg=True))

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 125557 rows; 6494 cols; 260512 nonzeros; 6480 integer variables (3240 binary)
Coefficient ranges:
  Matrix [1e+00, 2e+02]
  Cost   [2e+01, 3e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+02]
Presolving model
125549 rows, 6486 cols, 260496 nonzeros  0s
Presolve: Infeasible

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0   

-1

In [0]:
print("Status:", LpStatus[model.status])

Status: Infeasible
